In [2]:
import pandas as pd
import numpy as np
import sys
import os
import subprocess as sps
import multiprocessing as mlp
from multiprocessing import Pool

In [3]:
libpath = '/gpfs/projects/KozakovGroup/mhc_learning/mhc-adventures/ilovemhc'
if libpath not in sys.path:
    sys.path.append(libpath)

In [4]:
import template_modelling
reload(template_modelling)

import plip_wrapper
reload(plip_wrapper)
from plip_wrapper import plip_read_pdb_complex, make_interaction_table, describe_interaction

In [5]:
table_path = '/gpfs/projects/KozakovGroup/mhc_learning/analysis/affinity_data/affinity_clean-15012019.csv'
mhc_seq_path = '/gpfs/projects/KozakovGroup/mhc_learning/analysis/allele_sequences_reduced.csv'
models_dir = 'models'
nproc = 4

In [6]:
sps.check_output('mkdir -p %s' % models_dir, shell=True)

''

In [7]:
aff_table = pd.read_csv(table_path, low_memory=False)

In [8]:
aff_small = aff_table.sample(n=1000)

In [9]:
aff_small.reset_index(drop=True, inplace=True)

In [14]:
set(aff_small.length)

{8, 9, 10, 11, 12, 13, 14}

In [15]:
aff_small.to_csv("affinity-toy.csv", index=False)

In [16]:
aff_small

,allele,peptide,length,binding,sign,units,ic50,assay_type,pseudo_custom,pseudo_nielsen,start,end,source,allele_orig,assay
0,HLA-A*03:01,ELYDTSPTK,9,Positive-High,=,nM,10.50,dissociation constant KD (~IC50),YFFAFMEEYQERNVAQQTDVDTLYIQIYRDAYYIWTKKWEAHEAQL...,YFAMYQENVAQTDVDTLYIIYRDYTWAELAYTWY,384.0,392.0,http://www.uniprot.org/uniprot/P18377,HLA-A*03:01,http://www.iedb.org/assay/1191962
1,HLA-A*03:01,AMCTNTFVLK,10,Positive-High,=,nM,20.30,dissociation constant KD (~IC50),YFFAFMEEYQERNVAQQTDVDTLYIQIYRDAYYIWTKKWEAHEAQL...,YFAMYQENVAQTDVDTLYIIYRDYTWAELAYTWY,578.0,587.0,http://www.uniprot.org/uniprot/P17763,HLA-A*03:01,http://www.iedb.org/assay/1886653
2,HLA-A*68:01,STDNAVYQCR,10,Positive-Intermediate,=,nM,81.30,dissociation constant KD (~IC50),YYFAFMEEYRNRNVAQQTDVDTLYIQMYRDAYYIWTKKWEAHVAQW...,YYAMYRNNVAQTDVDTLYIMYRDYTWAVWAYTWY,67.0,76.0,http://www.uniprot.org/uniprot/P24761,HLA-A*68:01,http://www.iedb.org/assay/1200883
3,HLA-A*02:01,ILPCPQINTL,10,Positive-Low,=,nM,1374.50,dissociation constant KD (~IC50),YFFAFMEEYGERKVAHQTHVDTLYVQRYHYAYYIWTKKWEAHVAQL...,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY,45.0,54.0,http://www.uniprot.org/uniprot/P25212,HLA-A*02:01,http://www.iedb.org/assay/1579885;http://www.i...
4,HLA-B*57:01,TSVTPRPSW,9,Positive,NaN,NaN,NaN,ligand presentation,YYFAFMEEYGERNMASQTYENIAYIQVYDSAYYIWTQKWEARVAQL...,YYAMYGENMASTYENIAYIVYDSYTWAVLAYLWY,241.0,249.0,http://www.uniprot.org/uniprot/Q9Y2Y4,HLA-B*57:01,http://www.iedb.org/assay/3123693;http://www.i...
5,HLA-C*01:02,DSPAGKTNAL,10,Positive,NaN,NaN,NaN,ligand presentation,YFFSFGEEYREQKYRQQTDVSNLYLQWCDYAYYIWTQKWEAREAQR...,YFSGYREKYRQTDVSNLYLWCDYYTWAERAYTWY,1110.0,1119.0,http://www.uniprot.org/uniprot/Q8NI35,HLA-C*01:02,http://www.iedb.org/assay/3354883;http://www.i...
6,HLA-B*27:05,LLLLISLVY,9,Negative,>,nM,20000.00,dissociation constant KD (~EC50),YHFTFEEEYREQICAKQTDEDTLYLQNYHDAYYIWTQKWEARVAQL...,YHTEYREICAKTDEDTLYLNYHDYTWAVLAYEWY,7.0,15.0,http://www.uniprot.org/uniprot/Q01218,HLA-B*27:05,http://www.iedb.org/assay/1446412
7,HLA-B*44:03,AELAVILKF,9,Positive,NaN,NaN,NaN,ligand presentation,YYFTFKEEYREQISTNQTYENTAYIQRYDDAYYIWTQKWEARVAQL...,YYTKYREISTNTYENTAYIRYDDYTWAVLAYLSY,125.0,133.0,http://www.uniprot.org/uniprot/Q14980,HLA-B*44:03,http://www.iedb.org/assay/3188000
8,HLA-B*27:05,GRVFKVGEPTY,11,Positive,NaN,NaN,NaN,ligand presentation,YHFTFEEEYREQICAKQTDEDTLYLQNYHDAYYIWTQKWEARVAQL...,YHTEYREICAKTDEDTLYLNYHDYTWAVLAYEWY,100.0,110.0,http://www.uniprot.org/uniprot/Q8IWV8,HLA-B*27:05,http://www.iedb.org/assay/2837066;http://www.i...
9,HLA-B*27:01,QRIFEPPPPK,10,Positive,NaN,NaN,NaN,ligand presentation,YHFTFEEEYREQICAKQTYENTAYLQNYHDAYYIWTQKWEARVAQL...,YHTEYREICAKTYENTAYLNYHDYTWAVLAYEWY,398.0,407.0,http://www.uniprot.org/uniprot/O43143,HLA-B*27:01,http://www.iedb.org/assay/3255190


In [17]:
mhc_seq_table = pd.read_csv(mhc_seq_path, sep=' ')

In [18]:
mhc_seq_dict = dict(zip(mhc_seq_table.allele_short, mhc_seq_table.sequence))

In [19]:
modeller = template_modelling.TemplateModeller()

In [21]:
set(modeller.pdb_table.pep_len)

{8, 9, 10, 11, 12, 13, 14}

In [ ]:
resi_map = []
for i, row in modeller.pdb_table.iterrows():
    old = map(int, row.resi_orig.split(','))
    new = map(int, row.resi_aln.split(','))
    resi_map.append((row.pdb, dict(zip(old, new))))
    
resi_map = dict(resi_map)

In [ ]:
def run(pid):
    detailed_all = []
    general_all = {}
    pdb_list = []
    
    print pid
    
    counter = -1
    for rowi, row in aff_small.iterrows():
        counter += 1
        if counter % nproc != pid:
            continue
        
        pdb_name = '%06i.pdb' % rowi
        allele = row.allele
        peptide = row.peptide
        allele_seq = mhc_seq_dict[allele]

        save_path = os.path.join(models_dir, pdb_name)
        template, _, nmin, psf = modeller.create_model(save_path, allele_seq, peptide)
        cur_resi_map = resi_map[template]

        inter = plip_read_pdb_complex(nmin)
        detailed = make_interaction_table(inter)
        
        general = describe_interaction(inter)
        general['bsite_renum'] = [cur_resi_map[int(x[:-1])] for x in general['bsite_orig']]
        general['contacts_renum'] = [cur_resi_map[int(x[:-1])] for x in general['contacts_orig']]

        struct_id = os.path.basename(nmin)
        detailed['pdb'] = struct_id

        pdb_list.append(pdb_name)
        detailed_all.append(detailed)
        general_all[struct_id] = general

    general_all = pd.DataFrame(general_all).transpose()
    detailed_all = pd.concat(detailed_all)
    detailed_all.reset_index(drop=True, inplace=True)
    
    return pdb_list, detailed_all, general_all

In [ ]:
pool = Pool(nproc)
output = pool.map(run, range(nproc))

In [ ]:
zipped = zip(*output)

pdb_list = reduce(lambda x,y: x+y, zipped[0])

detailed_all = pd.concat(zipped[1]).reset_index(drop=True)

general_all = pd.concat(zipped[2]).reset_index()

In [ ]:
with open('models.list', 'w') as f:
    f.write('\n'.join(pdb_list) + '\n')

detailed_all.to_csv('detailed_interaction.csv', index=False)

general_all.to_csv('general_interaction.csv', index=False)